# 05 - Agentic AML Investigation Pipeline

Hybrid 2-Agent architecture for explainable money laundering detection:

**Architecture:**
```
┌─────────────────────────────────────────────────────────────────┐
│                    COORDINATOR AGENT                            │
│    (Plans investigation, selects tools, gathers evidence)       │
│                                                                 │
│    Tools: get_fraud_score, retrieve_similar_cases,             │
│           explain_prediction, get_network_context               │
└─────────────────────────────────────────────────────────────────┘
                              │
                              ▼ Evidence Package
┌─────────────────────────────────────────────────────────────────┐
│                     ANALYST AGENT                               │
│     (Reviews evidence, produces verdict + recommendation)       │
│                                                                 │
│    Output: Risk Level, Confidence, Recommendation, Reasoning    │
└─────────────────────────────────────────────────────────────────┘
```

**Key Features:**
- ReAct (Reason-Act-Observe) pattern for autonomous investigation
- Dynamic tool selection based on evidence needs
- Iterative evidence gathering
- Structured verdicts with confidence scores

## 1. Setup & Imports

In [1]:
import sys
sys.path.append('..')

import torch
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Check device
device = 'cpu'
print(f"Device: {device}")
print("Imports complete!")

Device: cpu
Imports complete!


## 2. Load Components

In [2]:
# Load graph data
from src.data.graph_builder import GraphBuilder

builder = GraphBuilder('../data/elliptic_bitcoin_dataset')
data = builder.build()

print(f"Graph loaded: {data.num_nodes:,} nodes, {data.num_edges:,} edges")

Loading Elliptic dataset...
  Features: 203,769 nodes, 167 columns
  Labels: 203,769 entries
  Edges: 234,355 directed edges
  Node mapping: 203,769 unique nodes
  Normalizing features...
  Features shape: (203769, 166)
  Time steps: 1 to 49 (49 unique)
  Label distribution:
    unknown: 157,205 (77.15%)
    licit: 42,019 (20.62%)
    illicit: 4,545 (2.23%)
  Edge index shape: (2, 234355)

Temporal splits:
  Train: 34 time steps, 136,265 nodes (29,894 labeled)
  Val:   7 time steps, 30,680 nodes (7,829 labeled)
  Test:  8 time steps, 36,824 nodes (8,841 labeled)

Class weights: licit=0.5541, illicit=5.1226

PyG Data Object Created
Nodes: 203,769
Edges: 234,355
Features: 166
Train nodes: 29,894
Val nodes: 7,829
Test nodes: 8,841
Graph loaded: 203,769 nodes, 234,355 edges


In [3]:
# Load trained model
from src.models.graphsage import GraphSAGEModel

model = GraphSAGEModel(
    num_features=data.num_node_features,
    num_classes=2,
    hidden_dims=[256, 128],
    embedding_dim=64,
    dropout=0.3
)

checkpoint = torch.load('../baseline/checkpoints/graphsage_baseline.pt', map_location=device, weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"Model loaded! Test F1: {checkpoint['metrics']['test_f1']:.4f}")

Model loaded! Test F1: 0.4102


In [4]:
# Load case memory and FAISS index
from src.memory.case_store import CaseMemory
from src.retrieval.faiss_index import FAISSIndex

case_memory = CaseMemory.load('../case_memory')
faiss_index = FAISSIndex(embedding_dim=64)
faiss_index.load('../case_memory/faiss.index')

print(f"Case memory: {len(case_memory.cases)} cases")
print(f"FAISS index: {faiss_index.num_vectors} vectors")

Loaded 1000 cases from ../case_memory
Loaded FAISS index from ../case_memory/faiss.index: 1000 vectors
Case memory: 1000 cases
FAISS index: 1000 vectors


## 3. Initialize Agentic Pipeline

In [5]:
from src.agents import AMLOrchestrator

# Choose LLM provider: "ollama", "openai", or "anthropic"
LLM_PROVIDER = "ollama"  # Change as needed

orchestrator = AMLOrchestrator(
    model=model,
    data=data,
    case_memory=case_memory,
    faiss_index=faiss_index,
    llm_provider=LLM_PROVIDER,
    device=device,
    verbose=True,
    max_steps=5
)

print(f"\nAgentic Pipeline initialized!")
print(f"LLM Provider: {LLM_PROVIDER}")


Agentic Pipeline initialized!
LLM Provider: ollama


In [6]:
# Test LLM connection
if orchestrator.test_llm_connection():
    print("✅ LLM connection successful!")
else:
    print("❌ LLM connection failed. Check your provider settings.")
    print("   For Ollama: run 'ollama serve' in terminal")
    print("   For OpenAI/Anthropic: check API keys in .env file")

✅ LLM connection successful!


## 4. Find Suspicious Transactions

In [7]:
# Find high-risk unlabeled transactions
with torch.no_grad():
    logits, _ = model(data.x, data.edge_index)
    fraud_scores = torch.softmax(logits, dim=1)[:, 1].numpy()

labels = data.y.numpy()
unlabeled_mask = labels == -1
high_risk = np.where(unlabeled_mask & (fraud_scores > 0.7))[0]

print(f"Found {len(high_risk)} high-risk unlabeled transactions")
print(f"\nTop 10 suspicious transactions:")
for idx in high_risk[:10]:
    print(f"  Node {idx}: {fraud_scores[idx]*100:.1f}% fraud score")

Found 23899 high-risk unlabeled transactions

Top 10 suspicious transactions:
  Node 84: 97.8% fraud score
  Node 115: 99.2% fraud score
  Node 190: 84.0% fraud score
  Node 232: 81.3% fraud score
  Node 243: 94.9% fraud score
  Node 311: 75.6% fraud score
  Node 335: 100.0% fraud score
  Node 336: 100.0% fraud score
  Node 382: 99.9% fraud score
  Node 411: 85.2% fraud score


## 5. Run Agentic Investigation

Watch how the Coordinator Agent:
1. Plans what information to gather
2. Selects and invokes appropriate tools
3. Reasons about observations
4. Passes evidence to Analyst Agent

In [8]:
# Select a suspicious transaction to investigate
target_node = high_risk[0] if len(high_risk) > 0 else 12345

print(f"Selected Transaction #{target_node} for investigation")
print(f"ML Fraud Score: {fraud_scores[target_node]*100:.1f}%")

Selected Transaction #84 for investigation
ML Fraud Score: 97.8%


In [9]:
# Run full agentic investigation
result = orchestrator.investigate(target_node)


######################################################################
#  AGENTIC AML INVESTIGATION - Transaction #84
######################################################################

[PHASE 1: EVIDENCE GATHERING]

COORDINATOR: Starting investigation of Transaction #84

--- Step 1/5 ---
THOUGHT: To start investigating transaction #84, I need to gather some initial information about its fraud score. A low fraud score could indicate that the transaction is legitimate, while a high score might suggest illicit activity.
ACTION: get_fraud_score
OBSERVATION: Score: 0.9782, Risk: CRITICAL

--- Step 2/5 ---
THOUGHT: Since we already have the fraud score of 0.9782, which indicates a CRITICAL risk, I think it's essential to gather more information about this transaction before making any conclusions. Retrieving similar cases could provide valuable context and help us understand if this transaction is part of a larger pattern.
ACTION: retrieve_similar_cases
OBSERVATION: 5 illicit, 0 licit 

## 6. View Investigation Results

In [10]:
# Print the full verdict report
orchestrator.print_report(result)


TRANSACTION RISK ASSESSMENT REPORT

Transaction ID: #84
Assessment Date: 2026-02-18T22:44:02.265775

🔴 RISK LEVEL: CRITICAL
Confidence: 90%

RECOMMENDATION: FLAG_IMMEDIATE
Action: ESCALATE IMMEDIATELY to compliance team

KEY RISK FACTORS:
  • Very high fraud score (98%)
  • Majority of similar cases (5/5) were illicit

SIMILAR CASE ANALYSIS:
5 illicit and 0 licit similar cases found.

DETAILED REASONING:
Assessment based on 4 evidence sources. Fraud score: 98%. Similar cases suggest illicit pattern.




In [11]:
# View investigation steps
print("Investigation Steps:")
print("=" * 60)
for step in result.state.steps:
    print(f"\nStep {step.step_num}:")
    print(f"  Thought: {step.thought[:100]}...")
    print(f"  Action: {step.action}")
    print(f"  Result: {str(step.observation)[:100]}...")

Investigation Steps:

Step 1:
  Thought: To start investigating transaction #84, I need to gather some initial information about its fraud sc...
  Action: get_fraud_score
  Result: {'success': True, 'result': {'node_idx': 84, 'fraud_score': 0.9782, 'fraud_percentage': '97.8%', 'ri...

Step 2:
  Thought: Since we already have the fraud score of 0.9782, which indicates a CRITICAL risk, I think it's essen...
  Action: retrieve_similar_cases
  Result: {'success': True, 'result': {'query_node': 84, 'num_retrieved': 5, 'similar_cases': [{'rank': 1, 'ca...

Step 3:
  Thought: Since we have a high fraud score and multiple similar illicit cases, it's likely that this transacti...
  Action: get_network_context
  Result: {'success': True, 'result': {'node_idx': 84, 'out_degree': 1, 'in_degree': 1, 'total_degree': 2, 'nu...

Step 4:
  Thought: Since we already have the fraud score of 0.9782, which indicates a CRITICAL risk, and similar cases ...
  Action: get_network_context
  Result: {'success': 

In [12]:
# View collected evidence
print("Evidence Collected:")
print("=" * 60)
for tool_name, evidence in result.state.evidence.items():
    print(f"\n{tool_name}:")
    if isinstance(evidence, dict):
        for k, v in list(evidence.items())[:5]:
            print(f"  {k}: {v}")

Evidence Collected:

get_fraud_score:
  node_idx: 84
  fraud_score: 0.9782
  fraud_percentage: 97.8%
  risk_level: CRITICAL
  predicted_label: illicit

retrieve_similar_cases:
  query_node: 84
  num_retrieved: 5
  similar_cases: [{'rank': 1, 'case_id': 20, 'similarity': np.float32(0.4569), 'similarity_percentage': '45.7%', 'true_label': 'illicit', 'fraud_score': 0.9583, 'summary': 'Case 20: ILLICIT transaction with 96% fraud score'}, {'rank': 2, 'case_id': 132, 'similarity': np.float32(0.4365), 'similarity_percentage': '43.7%', 'true_label': 'illicit', 'fraud_score': 0.9704, 'summary': 'Case 132: ILLICIT transaction with 97% fraud score'}, {'rank': 3, 'case_id': 36, 'similarity': np.float32(0.3943), 'similarity_percentage': '39.4%', 'true_label': 'illicit', 'fraud_score': 0.9709, 'summary': 'Case 36: ILLICIT transaction with 97% fraud score'}, {'rank': 4, 'case_id': 80, 'similarity': np.float32(0.3803), 'similarity_percentage': '38.0%', 'true_label': 'illicit', 'fraud_score': 0.9926, '

## 7. Batch Investigation

In [13]:
# Investigate top 3 suspicious transactions
batch_results = orchestrator.batch_investigate(
    node_indices=high_risk[:3].tolist(),
    save_results=False
)


[Batch Progress: 1/3]

######################################################################
#  AGENTIC AML INVESTIGATION - Transaction #84
######################################################################

[PHASE 1: EVIDENCE GATHERING]

COORDINATOR: Starting investigation of Transaction #84

--- Step 1/5 ---
THOUGHT: Before starting the investigation, I need to get an initial fraud score for transaction #84. This will give me a baseline understanding of how likely it is that this transaction is illicit.
ACTION: get_fraud_score
OBSERVATION: Score: 0.9782, Risk: CRITICAL

--- Step 2/5 ---
THOUGHT: Since the fraud score is already high at 0.9782, indicating a CRITICAL risk, I need to gather more information to confirm if this transaction is indeed illicit. Retrieving similar cases could provide context and help identify potential patterns or red flags.
ACTION: retrieve_similar_cases
OBSERVATION: 5 illicit, 0 licit matches

--- Step 3/5 ---
THOUGHT: Since we have a high fraud score

In [14]:
# Summary of all investigations
print("\nBatch Investigation Summary:")
print("=" * 70)
print(f"{'Node':<10} {'Risk Level':<12} {'Confidence':<12} {'Recommendation':<20}")
print("-" * 70)

for r in batch_results:
    v = r.verdict
    print(f"{r.node_idx:<10} {v.risk_level.value:<12} {v.confidence*100:.0f}%{'':<8} {v.recommendation.value:<20}")


Batch Investigation Summary:
Node       Risk Level   Confidence   Recommendation      
----------------------------------------------------------------------
84         CRITICAL     85%         FLAG_IMMEDIATE      
115        CRITICAL     99%         FLAG_IMMEDIATE      
190        CRITICAL     85%         FLAG_IMMEDIATE      


In [15]:
# Get overall statistics
summary = orchestrator.get_summary()
print("\nOverall Investigation Statistics:")
print("=" * 60)
for key, value in summary.items():
    print(f"{key}: {value}")


Overall Investigation Statistics:
total_investigations: 4
risk_distribution: {'CRITICAL': 4, 'HIGH': 0, 'MEDIUM': 0, 'LOW': 0}
recommendation_distribution: {'FLAG_IMMEDIATE': 4, 'INVESTIGATE': 0, 'MONITOR': 0, 'CLEAR': 0}
average_confidence: 89.8%
average_duration_seconds: 16.04
high_risk_transactions: [np.int64(84), 84, 115, 190]


## 8. Compare: RAG Pipeline vs Agentic Pipeline

| Aspect | RAG Pipeline | Agentic Pipeline |
|--------|-------------|------------------|
| **Flow** | Fixed sequence | Dynamic planning |
| **Tool Selection** | Always runs all | Based on needs |
| **Iteration** | Single pass | Can loop back |
| **Output** | Raw LLM text | Structured verdict |
| **Confidence** | Not provided | Explicit score |
| **Recommendation** | Manual | Automatic |

## 9. Save Results

In [18]:
# Reload the module to pick up fixes and save result
import importlib
import src.agents.orchestrator as orch_module
importlib.reload(orch_module)

# Manual save with numpy handling
import json
import numpy as np

def convert_numpy(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {k: convert_numpy(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy(item) for item in obj]
    return obj

output_file = f'../case_memory/agentic_investigation_{target_node}.json'
with open(output_file, 'w') as f:
    json.dump(convert_numpy(result.to_dict()), f, indent=2)
print(f"Investigation saved to: {output_file}")

Investigation saved to: ../case_memory/agentic_investigation_84.json


## 10. Summary

The **Hybrid 2-Agent Architecture** provides:

1. **Autonomous Investigation** - Coordinator Agent plans and executes investigation
2. **Dynamic Tool Selection** - Tools invoked based on evidence needs
3. **Structured Verdicts** - Risk level, confidence, and recommendations
4. **Explainable Decisions** - Full reasoning chain and key factors
5. **Audit Trail** - Complete investigation history for compliance

In [19]:
print("="*60)
print("AGENTIC AML PIPELINE SUMMARY")
print("="*60)
print(f"\n🤖 Agents:")
print(f"   • Coordinator: Plans investigation, invokes tools")
print(f"   • Analyst: Produces verdicts with confidence scores")
print(f"\n🔧 Tools Available:")
for tool_name in orchestrator.tools.tool_names:
    print(f"   • {tool_name}")
print(f"\n📊 Investigation Stats:")
print(f"   • Total investigations: {len(orchestrator._investigation_history)}")
summary = orchestrator.get_summary()
if 'high_risk_transactions' in summary:
    print(f"   • High-risk flagged: {len(summary['high_risk_transactions'])}")
print(f"\n✅ Pipeline Status: READY")

AGENTIC AML PIPELINE SUMMARY

🤖 Agents:
   • Coordinator: Plans investigation, invokes tools
   • Analyst: Produces verdicts with confidence scores

🔧 Tools Available:
   • get_fraud_score
   • retrieve_similar_cases
   • explain_prediction
   • get_network_context
   • lookup_case
   • get_transaction_features

📊 Investigation Stats:
   • Total investigations: 4
   • High-risk flagged: 4

✅ Pipeline Status: READY
